In [7]:
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.utils import to_categorical
import networkx as nx

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

In [2]:
def get_labels(g):
    road_graph = nx.read_gml(g)

In [6]:
def preprocess(image):
    map_image = load_img(image)
    map_array = img_to_array(map_image) / 255.0  # Normalize pixel values
    grayscale_map = map_array.mean(axis=-1)[:, :, None]
    
    road_mask = grayscale_map > 0.5
    road_segments = skimage.morphology.label(road_mask)
    return grayscale_map, road_segments

In [5]:
def cnn_model(map, road_graph):
    model = Sequential([
        Conv2D(32, (3, 3), activation="relu", input_shape=(map.shape[0], map.shape[1], 1)),
        MaxPooling2D((2, 2)),
        Conv2D(64, (3, 3), activation="relu"),
        MaxPooling2D((2, 2)),
        Flatten(),
        Dense(128, activation="relu"),
        Dense(road_graph.number_of_nodes(), activation="softmax")
    ])
    model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
    return model

In [8]:
def train_cnn(road_segment):
    training_images = []
    training_labels = []
    for segment in road_segments.ravel():
        if segment > 0:  # Consider only segments with potential road pixels
            segment_mask = (road_segments == segment).astype(np.float32)
            training_images.append(segment_mask)        
            segment_label = to_categorical([list(road_graph.neighbors(n)) for n in road_graph.nodes() if road_graph.nodes[n]["segment"] == segment])[0]
            training_labels.append(segment_label)
    model.fit(np.array(training_images), np.array(training_labels), epochs=10)
    return model

In [ ]:
def predict(new_map, model):
    new_map_image = load_img(new_map)
    new_map_array = img_to_array(new_map_image) / 255.0
    new_grayscale_map = new_map_array.mean(axis=-1)[:, :, None]
    predicted_segments = model.predict(np.array([new_grayscale_map]))[0]
    predicted_segment_labels = np.argmax(predicted_segments, axis=-1)

In [ ]:
def make_network(predicted_segment_labels)
    extracted_network = nx.Graph()
    for i, segment_label in enumerate(predicted_segment_labels):
        if segment_label > 0:  
            extracted_network.add_node(i, segment=segment_label)